In [33]:
#basic CVA modelling

#Get EPE's from pricing modules

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import CubicSpline

dfExp = pd.read_csv('exposure_results.csv')

EPE = dfExp['EPE'].values
ENE = dfExp['ENE'].values

#print(EPE)
#print(ENE)

recovery = 0.4

#Default probabilities - CDS Spreads

data = pd.read_csv("CDS_data.csv")
time = data['Tenor'].values
CDS = data['CDS_Spread'].values
haz_Rates = data['Hazard_rate'].values

cs = CubicSpline(time, CDS)

timesteps = np.arange(0, 10.05, 0.25)
df = np.linspace(1, 0.9, len(timesteps))
spread = cs(timesteps)
survival = np.exp((-spread * timesteps)/(1 - recovery))
default = 1 - survival

#print(default)
#Default probabilities - piecewise hazard rates
    
def gethazardrate(t, time_intervals, haz_Rates):
    for i in range(len(time_intervals) - 1):
        if t >= time_intervals[i] and t < time_intervals[i + 1]:
            return haz_Rates[i]
    return haz_Rates[-1]  

hazr = np.zeros(len(timesteps))
survival_haz = np.zeros(len(timesteps))

for i in range(len(timesteps)):
    hazr[i] = gethazardrate(timesteps[i], time, haz_Rates)

#print(hazr)
survival_haz[0] =  1

for i in range(1, len(timesteps)):
    dt = timesteps[i] - timesteps[i-1]
    survival_haz[i] = np.exp(-hazr[i] * dt) * survival_haz[i-1]
    
#CVA from CDS spreads

#print(len(EPE))
#print(len(survival_haz))
#print(survival_haz)

CVA = 0

for i in range(len(timesteps) - 1):
    CVA += EPE[i] * (default[i + 1] - default[i]) * df[i] * (1 - recovery)
    
print(CVA)

#Wrong Way Risk Calculation



5.100530726574605


In [32]:
data = pd.read_csv("EPE.csv")
time = data['Time'].values
CDS = data['EPE'].values

